In [212]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_load = "Resources/schools_complete.csv"
student_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_load)
student_data = pd.read_csv(student_load)

# Combine the data into a single dataset
school_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [213]:
total_schools = school_data["school_name"].count()
print(total_schools)

total_students = student_data["Student ID"].count()
print(total_students)

total_budget = school_data["budget"].sum()
print(total_budget)

avg_math = student_data["math_score"].mean()
print(avg_math)

avg_reading = student_data["reading_score"].mean()
print(avg_reading)


passing_math = school_complete.query("math_score>70")["School ID"].count()/total_students * 100
passing_reading = school_complete.query("reading_score>70")["School ID"].count()/total_students * 100
print(passing_math)
print(passing_reading)

overall = (avg_math + avg_reading)/2
print(overall)



15
39170
24649428
78.98537145774827
81.87784018381414
72.39213683941792
82.97166198621395
80.43160582078121


In [214]:
results_df = pd.DataFrame({'Total Schools': total_schools,
                            'Total Students': total_students,
                            'Total Budget' : total_budget,
                            'Average Math Score' : avg_math,
                            'Average Reading Score' : avg_reading,
                            '% Passing Math' : passing_math,
                            '% Passing Reading' : passing_reading,
                            '% Overall Passing Rate' : overall
                           }, index=[0])
results_df




,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,72.392137,82.971662,80.431606


In [215]:
school_student_df = pd.merge(student_data,school_data,how='left',on='school_name')
school_student_df.head()

school_sum = pd.DataFrame({'Total Students' : 
    school_student_df.groupby(['school_name','type','budget'])['Student ID'].count(),
    'Average Math Score' : ( school_student_df.groupby(['school_name','type','budget'])['math_score'].sum()
    /  school_student_df.groupby(['school_name','type','budget'])['math_score'].count()),
    'Average Reading Score' : ( school_student_df.groupby(['school_name','type','budget'])['reading_score'].sum()
    /  school_student_df.groupby(['school_name','type','budget'])['reading_score'].count()),
    '% Passing Math' : ((school_student_df[school_student_df['math_score']>=70].groupby(['school_name','type','budget'])['Student ID'].count()
    / school_student_df.groupby(['school_name','type','budget'])['Student ID'].count())*100).map("{:.6f}".format),
    '% Passing Reading' : ((school_student_df[school_student_df['reading_score']>=70].groupby(['school_name','type','budget'])['Student ID'].count()
    / school_student_df.groupby(['school_name','type','budget'])['Student ID'].count())*100).map("{:.6f}".format)
                                      
                                       }).reset_index()
#Find Per Student Budget and add it to the dataframe
school_sum['Per Student Budget'] = (school_sum['budget']/school_sum['Total Students']).map("${:.2f}".format)                                            
#Find % Overall Passing Rate and add it to the dataframe
school_sum['% Overall Passing Rate'] = school_sum[['% Passing Math','% Passing Reading']].astype(float).mean(axis=1)
#Set school_name as index
school_sum = school_sum.set_index('school_name')
#using map format budget to include $
school_sum['budget'] = school_sum['budget'].map("${:,.2f}".format)
school_sum

,type,budget,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Per Student Budget,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"$3,124,928.00",4976,77.048432,81.033963,66.680064,81.933280,$628.00,74.306672
Cabrera High School,Charter,"$1,081,356.00",1858,83.061895,83.975780,94.133477,97.039828,$582.00,95.586652
Figueroa High School,District,"$1,884,411.00",2949,76.711767,81.158020,65.988471,80.739234,$639.00,73.363853
Ford High School,District,"$1,763,916.00",2739,77.102592,80.746258,68.309602,79.299014,$644.00,73.804308
Griffin High School,Charter,"$917,500.00",1468,83.351499,83.816757,93.392371,97.138965,$625.00,95.265668
Hernandez High School,District,"$3,022,020.00",4635,77.289752,80.934412,66.752967,80.862999,$652.00,73.807983
Holden High School,Charter,"$248,087.00",427,83.803279,83.814988,92.505855,96.252927,$581.00,94.379391
Huang High School,District,"$1,910,635.00",2917,76.629414,81.182722,65.683922,81.316421,$655.00,73.500171
Johnson High School,District,"$3,094,650.00",4761,77.072464,80.966394,66.057551,81.222432,$650.00,73.639992


In [216]:
school_sum = school_sum.sort_values(by=['% Overall Passing Rate'], ascending=False)
top_df = school_sum.head(5)
top_df

,type,budget,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Per Student Budget,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,"$1,081,356.00",1858,83.061895,83.975780,94.133477,97.039828,$582.00,95.586652
Thomas High School,Charter,"$1,043,130.00",1635,83.418349,83.848930,93.272171,97.308869,$638.00,95.290520
Pena High School,Charter,"$585,858.00",962,83.839917,84.044699,94.594595,95.945946,$609.00,95.270271
Griffin High School,Charter,"$917,500.00",1468,83.351499,83.816757,93.392371,97.138965,$625.00,95.265668
Wilson High School,Charter,"$1,319,574.00",2283,83.274201,83.989488,93.867718,96.539641,$578.00,95.203679


In [217]:
bottom_df = school_sum.tail(5)
bottom_df = bottom_df.sort_values(by=['% Overall Passing Rate'])
bottom_df

,type,budget,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Per Student Budget,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,"$2,547,363.00",3999,76.842711,80.744686,66.366592,80.220055,$637.00,73.293323
Figueroa High School,District,"$1,884,411.00",2949,76.711767,81.158020,65.988471,80.739234,$639.00,73.363853
Huang High School,District,"$1,910,635.00",2917,76.629414,81.182722,65.683922,81.316421,$655.00,73.500171
Johnson High School,District,"$3,094,650.00",4761,77.072464,80.966394,66.057551,81.222432,$650.00,73.639992
Ford High School,District,"$1,763,916.00",2739,77.102592,80.746258,68.309602,79.299014,$644.00,73.804308


In [218]:
math_scores = pd.DataFrame({'9th' : ((school_student_df[school_student_df['grade']=="9th"].groupby(['school_name'])['math_score'].sum()
    /school_student_df[school_student_df['grade']=="9th"].groupby(['school_name'])['Student ID'].count())),
    '10th' : ((school_student_df[school_student_df['grade']=="10th"].groupby(['school_name'])['math_score'].sum()
    /school_student_df[school_student_df['grade']=="10th"].groupby(['school_name'])['Student ID'].count())),
    '11th' : ((school_student_df[school_student_df['grade']=="11th"].groupby(['school_name'])['math_score'].sum()
    /school_student_df[school_student_df['grade']=="11th"].groupby(['school_name'])['Student ID'].count())),
    '12th' : ((school_student_df[school_student_df['grade']=="12th"].groupby(['school_name'])['math_score'].sum()
    /school_student_df[school_student_df['grade']=="12th"].groupby(['school_name'])['Student ID'].count())),
    })

math_scores

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [219]:
reading_scores = pd.DataFrame({'9th' : ((school_student_df[school_student_df['grade']=="9th"].groupby(['school_name'])['reading_score'].sum()
    /school_student_df[school_student_df['grade']=="9th"].groupby(['school_name'])['Student ID'].count())),
    '10th' : ((school_student_df[school_student_df['grade']=="10th"].groupby(['school_name'])['reading_score'].sum()
    /school_student_df[school_student_df['grade']=="10th"].groupby(['school_name'])['Student ID'].count())),
    '11th' : ((school_student_df[school_student_df['grade']=="11th"].groupby(['school_name'])['reading_score'].sum()
    /school_student_df[school_student_df['grade']=="11th"].groupby(['school_name'])['Student ID'].count())),
    '12th' : ((school_student_df[school_student_df['grade']=="12th"].groupby(['school_name'])['reading_score'].sum()
    /school_student_df[school_student_df['grade']=="12th"].groupby(['school_name'])['Student ID'].count())),
                                       })
reading_scores.index.name = " "
reading_scores

,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [220]:
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

school_spending =school_sum
school_spending["Spending Ranges (Per Student)"] = pd.cut(
    school_spending["Per Student Budget"].str.strip("$").astype(float),spending_bins
    ,labels= group_names)


school_spend_scores = pd.DataFrame(
                               {'Average Math Score': school_spending.groupby("Spending Ranges (Per Student)")['Average Math Score'].mean(),
                                'Average Reading Score': school_spending.groupby("Spending Ranges (Per Student)")['Average Reading Score'].mean()                              
                               })
school_spend_scores['% Passing Math'] = pd.to_numeric(school_spending['% Passing Math']).groupby(school_spending["Spending Ranges (Per Student)"]).mean()
school_spend_scores['% Passing Reading'] = pd.to_numeric(school_spending['% Passing Reading']).groupby(school_spending["Spending Ranges (Per Student)"]).mean()
school_spend_scores['% Overall Passing Rate'] = school_spending.groupby("Spending Ranges (Per Student)")['% Overall Passing Rate'].mean()


school_spend_scores

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065573
$615-645,79.079225,81.891436,75.668212,86.106570,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


In [233]:
school_sum_size = school_sum.copy()

size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_sum_size['Total Students'] = pd.to_numeric(school_sum['Total Students'])

#Cut the school size into the bins we define above 
school_sum_size['School Size'] = pd.cut(school_sum['Total Students'],
                                                                  bins, labels=group_names)

#selecte columns for dataframe
school_sum_size = school_sum_size[['Average Math Score', 'Average Reading Score',
                                          '% Passing Math', '% Passing Reading', '% Overall Passing Rate',
                                          'School Size']]

#group by size bins and and take the aveage of scores
school_sum_df = school_sum_size.groupby('School Size').mean()


school_sum_df

,Average Math Score,Average Reading Score,% Overall Passing Rate
School Size,,,
Small (<1000),83.821598,83.929843,94.824831
Medium (1000-2000),81.176821,82.933187,87.983105
Large (2000-5000),77.063340,80.919864,73.761993


In [227]:
school_type = school_sum.copy()

#select columns
school_type = school_sum[['type','Average Math Score', 'Average Reading Score',
                                          '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
                                          
#group by school type and average columns
school_type_df = school_sum.groupby('type').mean()

#display dataframe
school_type_df

,Total Students,Average Math Score,Average Reading Score,% Overall Passing Rate
type,,,,
Charter,1524.250000,83.473852,83.896421,95.103660
District,3853.714286,76.956733,80.966636,73.673757
